Import libraries for data cleaning 

In [16]:
import csv
from tokenize import String

from db_connection import get_db_engine
from db_connection import execute_query
import numpy as np
import pandas as pd
import os
import re
import pandas.io.sql as psql
import psycopg2 as ps

Reading CSV file

In [17]:
df_the_oscar_awards = pd.read_csv(r"C:\Users\ricca\Desktop\CSV\the_oscar_awards.csv\the_oscar_awards.csv")

Top records of CSV file: actors.csv

In [18]:
df_the_oscar_awards

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10884,2023,2024,96,WRITING (Original Screenplay),Written by Celine Song,Past Lives,False
10885,2023,2024,96,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10886,2023,2024,96,HONORARY AWARD,"To Angela Bassett, who has inspired audiences ...",NaN,True
10887,2023,2024,96,HONORARY AWARD,"To Mel Brooks, for his comedic brilliance, pro...",NaN,True


Checking for total dimension of the csv

In [19]:
df_the_oscar_awards.shape

(10889, 7)

In [20]:
df_the_oscar_awards.columns

Index(['year_film', 'year_ceremony', 'ceremony', 'category', 'name', 'film',
       'winner'],
      dtype='object')

Check info about column type

In [21]:
df_the_oscar_awards.dtypes

year_film         int64
year_ceremony     int64
ceremony          int64
category         object
name             object
film             object
winner             bool
dtype: object

Correction of data type of the records

In [22]:
df_the_oscar_awards['year_film'] = df_the_oscar_awards['year_film'].astype('Int64')
df_the_oscar_awards['year_ceremony'] = df_the_oscar_awards['year_ceremony'].astype('Int64')
df_the_oscar_awards['ceremony'] = df_the_oscar_awards['ceremony'].astype('Int64')
df_the_oscar_awards['category'] = df_the_oscar_awards['category'].fillna(pd.NA).astype('string')
df_the_oscar_awards['name'] = df_the_oscar_awards['name'].fillna(pd.NA).astype('string')
df_the_oscar_awards['film'] = df_the_oscar_awards['film'].fillna(pd.NA).astype('string')
df_the_oscar_awards['winner'] = df_the_oscar_awards['winner'].fillna(pd.NA).astype('bool')

Check if there are duplicates

In [23]:
df_the_oscar_awards.duplicated().sum()


np.int64(7)

In [24]:
df_the_oscar_awards.drop_duplicates(inplace = True)
df_the_oscar_awards.duplicated().sum()

np.int64(0)

Check the missing values

In [25]:
df_the_oscar_awards.isnull().sum()

year_film          0
year_ceremony      0
ceremony           0
category           0
name               4
film             318
winner             0
dtype: int64

Dropping records with missing value 

In [26]:
df_the_oscar_awards.dropna(subset=["film", 'winner'], inplace=True)
df_the_oscar_awards.isnull().sum()

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
winner           0
dtype: int64

In [27]:
df_the_oscar_awards.shape

(10564, 7)

Connecting to PostgreSQL database

In [28]:
engine = get_db_engine()
sql = \
"""CREATE TABLE IF NOT EXISTS the_oscar_awards (
        year_film INTEGER CHECK (year_film>=1870),
        year_ceremony INTEGER CHECK (year_ceremony>=1870),
        ceremony INTEGER CHECK (ceremony>0),
        category TEXT CHECK (category<>''),
        name TEXT CHECK (name<>''),
        film TEXT CHECK (film<>''),
        winner BOOLEAN NOT NULL DEFAULT FALSE
        
        PRIMARY KEY (year_ceremony, category, name, film)
        );
"""
execute_query(sql)

Error (psycopg2.errors.SyntaxError) ERRORE:  errore di sintassi a o presso "("
LINE 10:         PRIMARY KEY (year_ceremony, category, name, film)
                             ^

[SQL: CREATE TABLE IF NOT EXISTS the_oscar_awards (
        year_film INTEGER CHECK (year_film>=1870),
        year_ceremony INTEGER CHECK (year_ceremony>=1870),
        ceremony INTEGER CHECK (ceremony>0),
        category TEXT CHECK (category<>''),
        name TEXT CHECK (name<>''),
        film TEXT CHECK (film<>''),
        winner BOOLEAN NOT NULL DEFAULT FALSE
        
        PRIMARY KEY (year_ceremony, category, name, film)
        );
]
(Background on this error at: https://sqlalche.me/e/20/f405)


Population of the table using .to_sql function

In [29]:
df_the_oscar_awards.to_sql("the_oscar_awards", engine, if_exists="append", index=False)

564